In [29]:
# clear all the variables
%reset -f
import numpy as np
import pandas as pd
import sys
import graphlab
import os
from os import walk
import scipy.io
import xlrd, os, fnmatch
import pandas as pd
import tensorflow as tf
import plotly.plotly as py
import plotly.graph_objs as go
from scipy.spatial.distance import cdist
# Path variable (where the data is located)
myPath = './data/' 
# Append the path variable to existing search path
sys.path.append(myPath)
# Get the file information in the directory
file_list = []
ignore_list = ['pilot']
for root, dirs, files in os.walk(myPath):
    print files
    if 'pilot' in dirs:
        dirs[:]=[]
    for filename in files:
        if fnmatch.fnmatch(filename.lower(), "*csv"):
            file_list.append(os.path.join(root, filename))

['4_12Nov2018_20-00.csv', '1_12Nov2018_16-25.txt', '3_12Nov2018_18-35.txt', '2_12Nov2018_17-35.csv', '4_12Nov2018_20-00.txt', '3_12Nov2018_18-35.csv', '1_12Nov2018_16-25.csv', '2_12Nov2018_17-35.txt']


In [30]:
file_list

['./data/4_12Nov2018_20-00.csv',
 './data/2_12Nov2018_17-35.csv',
 './data/3_12Nov2018_18-35.csv',
 './data/1_12Nov2018_16-25.csv']

In [31]:
data = graphlab.SFrame.read_csv(file_list[0])
temp = data
for i in range(1,len(file_list)):
    temp = graphlab.SFrame.read_csv(file_list[i])
    data = data.append(temp)
len(data)

Finished parsing file /Users/supadhy6/Documents/GitHub/spatio_temporal_information_processing/RSVP_visual_now/data/4_12Nov2018_20-00.csv

Parsing completed. Parsed 100 lines in 0.038139 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[str,int,int,str,float,int,str,str,float,float,array,int,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /Users/supadhy6/Documents/GitHub/spatio_temporal_information_processing/RSVP_visual_now/data/4_12Nov2018_20-00.csv

Parsing completed. Parsed 420 lines in 0.006269 secs.

...

Parsing completed. Parsed 100 lines in 0.008931 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[str,int,int,str,float,int,str,str,float,float,array,int,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /Users/supadhy6/Documents/GitHub/spatio_temporal_information_processing/RSVP_visual_now/data/1_12Nov2018_16-25.csv

Parsing completed. Parsed 450 lines in 0.007065 secs.

1710

In [32]:
data.show()

Canvas is accessible via web browser at the URL: http://localhost:52261/index.html
Opening Canvas in default web browser.


In [45]:
from plotly import tools
trace = [[]]*2
colors = ['darkviolet','darkgreen','orange']
for i,cuePos in enumerate(data['cueSpatialPos1'].unique()):
    cuePosdata = data[data['cueSpatialPos1']==cuePos]
    temp1 = []
    for j,ecc in enumerate(cuePosdata['cueEccentricity1'].unique().sort()):
        ecc_data = cuePosdata[cuePosdata['cueEccentricity1']==ecc]
        temp1.append(go.Histogram(x = ecc_data['responsePosRelative0'].to_numpy(), name = str(ecc) + 'deg', opacity = 0.5, marker = dict(color = colors[j])))
    trace[i] = temp1

if data['cueSpatialPos1'].unique()[0] == [-1.0,0.0]:
    titles = ('West','East')
else:
    titles = ('East','West')
    
fig = tools.make_subplots(rows=1, cols=2, subplot_titles = titles)
fig['layout'].update(barmode='group', title = 'performance in RSVP visual now task for different cue positions - '+str(len(data)/420)+' participants')
#fig['layout']['xaxis3'].update(title='position of reported item relative to the true item', range = [-2,2])
#fig['layout']['xaxis4'].update(title='position of reported item relative to the true item', range = [-2,2])
fig['layout']['xaxis1'].update(range=[-2,2],title='position of reported item relative to the true item')
fig['layout']['xaxis2'].update(range=[-2,2],title='position of reported item relative to the true item')
fig['layout']['yaxis1'].update(title='Frequency')
#fig['layout']['yaxis3'].update(title='Frequency')
for i in range(3):
    fig.append_trace(trace[0][i], 1, 1)
    fig.append_trace(trace[1][i], 1, 2)
    #fig.append_trace(trace[2][i], 2, 1)
    #fig.append_trace(trace[3][i], 2, 2)
py.iplot(fig, filename='RSVP_visual_now')

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]



In [35]:
cuePosdata['cueEccentricity1'].unique().sort()

dtype: int
Rows: 3
[2, 6, 10]

In [44]:
len(data)/420

4